In [1]:
import sys
sys.path.append('./../src')

# python's shit
from datetime import date, datetime, timedelta
import time

# someone elses shit
import finnhub
import pandas

# my shit
import config
from lib.pickle_jar import PickleJar

In [2]:
finnhub_client = finnhub.Client(api_key=config.api_keys['finnhub']['sandbox'])

In [3]:
# Time junk, just for testing?
now = time.time()
today = date.today()
yesterday = today + timedelta(days = -1)

In [4]:
def recommendation_trends_by_date(ticker, date):
    first_of_month = date.replace(day = 1)
    recommendations = finnhub_client.recommendation_trends(ticker)
    recommendation = [r for r in recommendations if datetime.strptime(r['period'], '%Y-%m-%d').date() == first_of_month]
    if (len(recommendation) > 0):
        return recommendation[0]
    return None

In [5]:
def company_earnings_by_date(ticker, date):
    reporting_period = timedelta(days = 90)
    earnings = finnhub_client.company_earnings(ticker)
    earning = [r for r in earnings if datetime.strptime(r['period'], '%Y-%m-%d').date() + reporting_period > date]
    if (len(earning) > 0):
        return earning[0]
    return None

Build a base data frame from the stock's low, open, close, high and volume for the given time period

In [6]:
def stock_candles_by_date(ticker, date):
    first_of_month_ts = int(datetime.combine(date.replace(day = 1), datetime.min.time()).timestamp())
    next_month = date.replace(day = 28) + timedelta(days = 4)
    last_of_month = next_month - timedelta(days = next_month.day)
    last_of_month_ts = datetime.combine(last_of_month, datetime.max.time()).timestamp()
    earliest = int(min(time.time(), last_of_month_ts))
    candles = finnhub_client.stock_candles('AAPL', '60', first_of_month_ts, earliest)
    return zip(candles['l'], candles['o'], candles['c'], candles['h'], candles['v'])

Construct a set of data frames that each hold:
- low
- open
- close
- high
- volume
- eps_actual (the actual EPS of the last relevant reporting period of the data frame's time period)
- eps_estimate (the estimated EPS, same as above)
- rec_strong_sell
- rec_sell
- rec_hold
- rec_buy
- rec_strong_sell (this and the above 4 fields are proportions of each rec cat of the total [0 - 1])

In [7]:
def construct_data_frames(ticker, date):
    trend = recommendation_trends_by_date(ticker, date)
    earnings = company_earnings_by_date(ticker, date)
    static_data = [
        earnings['actual'], earnings['estimate'],
        trend['strongSell'], trend['sell'], trend['hold'], trend['buy'], trend['strongBuy']
    ]
    base_frames = stock_candles_by_date(ticker, date)
    return [list(x) + static_data for x in base_frames]

In [8]:
jar = PickleJar()
data_frames_02_24_22 = jar.read_pickle_dataframe("{0}-{1}.technical.df.pkl".format('AAPL', today.strftime("%m-%d-%Y")))

In [9]:
data_frames_02_24_22.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,122.631000,123.000000,123.282900,123.492000,41,1.68,1.555857,5.5,5.5,39.5,113,62.5
1,123.182840,123.553500,123.837673,124.047714,85,1.68,1.555857,5.5,5.5,39.5,113,62.5
2,123.737162,124.109491,124.394943,124.605929,99,1.68,1.555857,5.5,5.5,39.5,113,62.5
3,124.293980,124.667983,124.480981,125.166655,22,1.68,1.555857,5.5,5.5,39.5,113,62.5
4,124.045392,124.418647,124.704810,124.916322,74,1.68,1.555857,5.5,5.5,39.5,113,62.5
